<a href="https://colab.research.google.com/github/selvaganesh19/Road-Traffic-Prediction-using-Machine-Learning/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import numpy as np
from comet_ml import start
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report, confusion_matrix

In [15]:
experiment = start(
    api_key="S7dshT9Bsag2XKtsWQV4RJp2L",
    project_name="road-traffic-prediction",
    workspace="selvaganesh19"
)

COMET WARNING: Ending the running experiment and creating a new Experiment because:
project_name doesn't match ('road-traffic-prediction' != 'road-traffic-prediction-using-machine-learning')
COMET ERROR: Failed to send stdout/stderr message batch, got 403 b'{"msg":"No more data will be logged to this Experiment since project has been deleted","code":403,"data":null,"sdk_error_code":7000022}'
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : neutral_gulf_458
COMET INFO:     url                   : https://www.comet.com/selvaganesh19/road-traffic-prediction-using-machine-learning/be1f38e5b4b945808ca402bb4b103bb9
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [4] : (0.85, 3.14)
COME

In [16]:
file_path = '/content/futuristic_city_traffic.csv'
traffic_data = pd.read_csv(file_path)
traffic_data = traffic_data.dropna(subset=['Traffic Density'])
traffic_data.fillna(traffic_data.mean(numeric_only=True), inplace=True)

In [17]:
# Label Encoding
label_encoders = {}
for column in ['City', 'Vehicle Type', 'Weather', 'Economic Condition', 'Day Of Week']:
    le = LabelEncoder()
    traffic_data[column] = le.fit_transform(traffic_data[column])
    label_encoders[column] = le


In [18]:
# Standard Scaling
scaler = StandardScaler()
numerical_features = ['Speed', 'Energy Consumption']
traffic_data[numerical_features] = scaler.fit_transform(traffic_data[numerical_features])


In [19]:

# Feature selection
X = traffic_data.drop(['Traffic Density', 'Is Peak Hour', 'Vehicle Type'], axis=1)
y_density = traffic_data['Traffic Density']
y_peak = traffic_data['Is Peak Hour']
y_vehicle = traffic_data['Vehicle Type']


In [20]:

# Train-test split
X_train_density, X_test_density, y_train_density, y_test_density = train_test_split(X, y_density, test_size=0.2, random_state=42)
X_train_peak, X_test_peak, y_train_peak, y_test_peak = train_test_split(X, y_peak, test_size=0.2, random_state=42)
X_train_vehicle, X_test_vehicle, y_train_vehicle, y_test_vehicle = train_test_split(X, y_vehicle, test_size=0.2, random_state=42)


In [21]:

# Log dataset details
experiment.log_dataset_hash(traffic_data)


In [22]:

# Log hyperparameters
hyper_params = {
    "learning_rate": 0.5,
    "steps": 100000,
    "batch_size": 50,
    "test_size": 0.2,
    "random_state": 42,
    "n_estimators": 100
}
experiment.log_parameters(hyper_params)

hidden_layer_size = 50
experiment.log_parameter("hidden_layer_size", hidden_layer_size)


In [23]:

# Model training
regressor = RandomForestRegressor(random_state=42)
regressor.fit(X_train_density, y_train_density)
classifier_peak = RandomForestClassifier(random_state=42)
classifier_peak.fit(X_train_peak, y_train_peak)
classifier_vehicle = RandomForestClassifier(random_state=42)
classifier_vehicle.fit(X_train_vehicle, y_train_vehicle)


RandomForestClassifier(random_state=42)

In [24]:

# Predictions
y_pred_density = regressor.predict(X_test_density)
y_pred_peak = classifier_peak.predict(X_test_peak)
y_pred_vehicle = classifier_vehicle.predict(X_test_vehicle)


In [25]:

# Log metrics
experiment.log_metric("mse_traffic_density", mean_squared_error(y_test_density, y_pred_density))
experiment.log_metric("accuracy_peak_hour", accuracy_score(y_test_peak, y_pred_peak))
experiment.log_metric("accuracy_vehicle_type", accuracy_score(y_test_vehicle, y_pred_vehicle))
experiment.log_metric("train_accuracy", 3.14, step=0)


In [26]:

# Log confusion matrices
experiment.log_confusion_matrix(y_test_peak, y_pred_peak, title="Peak Hour Confusion Matrix")
experiment.log_confusion_matrix(y_test_vehicle, y_pred_vehicle, title="Vehicle Type Confusion Matrix")


COMET ERROR: Error creating confusion matrix: 0; ignoring
COMET ERROR: Error creating confusion matrix: 0; ignoring


In [27]:

# Sample Prediction
sample_data = X.iloc[0].values.reshape(1, -1)
predicted_density = regressor.predict(sample_data)[0]
predicted_peak = classifier_peak.predict(sample_data)[0]
predicted_vehicle = classifier_vehicle.predict(sample_data)[0]
predicted_vehicle_decoded = label_encoders['Vehicle Type'].inverse_transform([predicted_vehicle])[0]

experiment.log_text("Sample Prediction")
experiment.log_text(f"Predicted Traffic Density: {predicted_density}")
experiment.log_text(f"Predicted Peak Hour: {'Yes' if predicted_peak == 1 else 'No'}")
experiment.log_text(f"Predicted Vehicle Type: {predicted_vehicle_decoded}")

print("\nPrediction:")
print(f"Predicted Traffic Density: {predicted_density}")
print(f"Predicted Peak Hour: {'Yes' if predicted_peak == 1 else 'No'}")
print(f"Predicted Vehicle Type: {predicted_vehicle_decoded}")

# End the experiment
experiment.end()

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(



Prediction:
Predicted Traffic Density: 0.454485
Predicted Peak Hour: No
Predicted Vehicle Type: Drone


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : organic_run_2265
COMET INFO:     url                   : https://www.comet.com/selvaganesh19/road-traffic-prediction/61296a7462b94ddf8612983c0574ad07
COMET INFO:   Metrics:
COMET INFO:     accuracy_peak_hour    : 0.9981225064538841
COMET INFO:     accuracy_vehicle_type : 0.923335680200266
COMET INFO:     mse_traffic_density   : 0.013680585425877259
COMET INFO:     train_accuracy        : 3.14
COMET INFO:   Others:
COMET INFO:     notebook_url : https://colab.research.google.com/notebook#fileId=12eWQgwJrUmTFL_NZ7Dm75g7KllBZMiPT
COMET INFO:   Parameters:
COMET INFO:     batch_size               : 50
COMET INFO:     bootstrap                : True
COMET